In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sec_edgar.data.preprocessing.preprocess_4form_files import pre_process_4form_archive_files
from sec_edgar.data.preprocessing.preprocess_master_idx import pre_process_master_idx_content
from sec_edgar.data.processing.process_4form_files import Process4FormFiles
from sec_edgar.data.processing.process_append_historical_prices import ProcessAppendHistoricalPrices
from sec_edgar.data.utils.cik_mapping_util import CikMappingUtil
from sec_edgar.stock_historical_data.av_historical_data_request import AVHistoricalDataRequest
import pandas as pd

from sec_edgar.utils.evaluation_metrics import PerformanceMetric01, PerformanceMetric02, PerformanceMetric03
from sec_edgar.utils.performance_evaluations import PerformanceEvaluations4Form, PerformanceEvaluations4FormYear

In [2]:
pd.set_option('display.width', 2500)      # max table width to display

In [3]:
## Loading of data set

In [4]:
data_set_name = "us_market_allderv_AandB_1019.py"
base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))

path_processed_datasets = base_path + '/Data/processed_datasets/'
file_name = path_processed_datasets + data_set_name + ".csv"

processed_4form_df = pd.read_csv(file_name, index_col=0)

In [5]:
print(file_name)
display(processed_4form_df)
display(processed_4form_df.columns.to_list())

C:\Users\andre\OneDrive\Documents\MathModels\sec_edgar/Data/processed_datasets/us_market_allderv_AandB_1019.py.csv


,CIK,Company_Names,Date_Filed,index_id,index_id_min,n_transactions,shares_percent_changes_mean,transactionSharesAdjust,date_ft_delta_mean,Filename,...,Shifted Price pct_change (10)(10 days 00:00:00),Shifted Price pct_change (10)(-10 days +00:00:00),Shifted Price pct_change (15)(15 days 00:00:00),Shifted Price pct_change (15)(-15 days +00:00:00),Shifted Price pct_change (21)(21 days 00:00:00),Shifted Price pct_change (21)(-21 days +00:00:00),Shifted Price pct_change (31)(31 days 00:00:00),Shifted Price pct_change (31)(-31 days +00:00:00),Shifted Price pct_change (42)(42 days 00:00:00),Shifted Price pct_change (42)(-42 days +00:00:00)
index,,,,,,,,,,,,,,,,,,,,,
48441,27904,DELTA AIR LINES INC /DE/,2010-01-04,[140546],140546,1,NaN,0.00,0 days 00:00:00,['edgar/data/27904/0001019687-10-000008.txt'],...,-0.071374,0.086661,-0.049117,0.075472,-0.043511,0.187556,0.140489,0.352113,0.174694,0.701164
107523,935703,DOLLAR TREE INC,2010-01-04,[578414 578415 578416 578417 578418],578414,5,0.250942,3544.61,3 days 00:00:00,['edgar/data/935703/0001127602-10-000234.txt'\...,...,0.005482,0.000612,-0.034031,0.014925,0.143033,0.017983,0.211652,0.053858,0.232375,-0.000204
48728,28412,COMERICA INC /NEW/,2010-01-04,[79905 79906 79907 79908 79909 79910 79911 799...,79905,13,0.018748,1152.00,5 days 00:00:00,['edgar/data/28412/0000028412-10-000001.txt'\n...,...,0.011134,0.068302,0.030569,0.152212,0.014648,0.167237,0.143071,0.266620,0.203143,0.213538
75418,72741,NORTHEAST UTILITIES,2010-01-04,[378340 378341 378342 378343],378340,4,0.146730,9435.00,0 days 00:00:00,['edgar/data/72741/0000072741-10-000004.txt'\n...,...,-0.040546,-0.000762,-0.051874,0.003069,0.028191,0.022868,0.074148,0.055285,0.109829,0.049897
97444,882835,ROPER INDUSTRIES INC,2010-01-04,[229120 229121 229122 229123],229120,4,-0.027286,0.00,5 days 00:00:00,['edgar/data/882835/0001209191-10-000310.txt'],...,-0.046095,-0.010336,-0.029718,-0.028972,0.074132,-0.028318,0.083082,-0.027359,0.165974,-0.048330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61641,49071,HUMANA INC,2019-12-31,[795842 795844 795845 795846 795847 795848 795...,795842,14,NaN,0.00,0 days 00:00:00,['edgar/data/49071/0000049071-19-000146.txt'\n...,...,-0.047265,-0.005206,-0.013514,0.024926,0.016773,0.056768,-0.189329,0.066934,-0.024946,0.088031
3960,1018724,AMAZON COM INC,2019-12-31,[334103],334103,1,NaN,0.00,4 days 00:00:00,['edgar/data/1018724/0000899243-19-030277.txt'],...,-0.029111,0.067372,0.101078,0.069387,0.138002,0.082767,-0.100004,0.033996,0.008402,0.076832
65091,54480,KANSAS CITY SOUTHERN,2019-12-31,[827929 827930 827932 827933 827934 827935 827...,827929,10,-0.001570,-285.00,11 days 00:00:00,['edgar/data/54480/0000054480-19-000247.txt'],...,0.036840,0.001612,0.076889,0.037715,0.063520,0.046904,-0.281522,0.047828,-0.197721,0.069084


['CIK',
 'Company_Names',
 'Date_Filed',
 'index_id',
 'index_id_min',
 'n_transactions',
 'shares_percent_changes_mean',
 'transactionSharesAdjust',
 'date_ft_delta_mean',
 'Filename',
 'Form_Type',
 'conversionOrExercisePrice',
 'date_ft_delta',
 'deemedExecutionDate',
 'directOrIndirectOwnership',
 'doc_name',
 'exerciseDate',
 'expirationDate',
 'file_url',
 'isDirector',
 'isOfficer',
 'isOther',
 'isTenPercentOwner',
 'issuerName',
 'issuerTradingSymbol',
 'officerTitle',
 'otherText',
 'periodOfReport',
 'quarter',
 'rptOwnerName',
 'securityTitle',
 'sharesOwnedFollowingTransaction',
 'shares_percent_changes',
 'signatureDate',
 'transactionAcquiredDisposedCode',
 'transactionCode',
 'transactionDate',
 'transactionFormType',
 'transactionPricePerShare',
 'transactionShares',
 'transactionTotalValue',
 'transaction_table',
 'transaction_type',
 'underlyingSecurityShares',
 'underlyingSecurityTitle',
 'underlyingSecurityValue',
 'valueOwnedFollowingTransaction',
 'year',
 'my_de

## Performance Metrics Evaluations

In [6]:
pe_4form = PerformanceEvaluations4Form(processed_4form_df, n_sub_set=5)

In [7]:
days = 5
column_label_eval = 'Price pct_change ({days})'.format(days=days)
shifted_columns_label_eval = 'Shifted Price pct_change ({days})(-{days} days +00:00:00)'.format(days=days)

In [8]:

# PerformanceMetric01 evaluation:
metrics_df_01, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric01,
                                        pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation: ")
display(metrics_df_01)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_02b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation: ")
display(metrics_df_02)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_03b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation: ")
display(metrics_df_03)
print("-" * 75)


PerformanceMetric01 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.003432,0.004719,0.002074,0.003452,0.003179,0.003736
benchmark,0.001884,0.00246,0.001698,0.001824,0.001633,0.001805
is_greater_prob,0.630588,0.729412,0.552941,0.623529,0.6,0.647059
is_pos_tse_ok,True,True,True,True,True,True
is_greater_prob_ok,True,True,True,True,True,True
is_performance_ok,True,True,True,True,True,True
Positive_tsa_std,0.007423,0.007044,0.004565,0.006185,0.007074,0.010755
benchmark_std,0.003258,0.002554,0.003142,0.002932,0.002957,0.004404
statistic,3.936642,2.77973,0.625293,2.191775,1.859299,1.532143
pvalue,0.000046,0.003221,0.266368,0.015164,0.032799,0.064161


---------------------------------------------------------------------------

PerformanceMetric02 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.003432,0.004719,0.002074,0.003452,0.003179,0.003736
benchmark,0.003088,0.004067,0.002626,0.003694,0.00297,0.00208
is_greater_prob,0.552941,0.611765,0.482353,0.505882,0.541176,0.623529
is_pos_tse_ok,True,True,False,False,True,True
is_greater_prob_ok,True,True,False,True,True,True
is_performance_ok,True,True,False,False,True,True
Positive_tsa_std,0.007423,0.007044,0.004565,0.006185,0.007074,0.010755
benchmark_std,0.010283,0.011227,0.00693,0.007751,0.007589,0.015471
statistic,0.559988,0.453346,-0.61375,-0.225617,0.186276,0.810399
pvalue,0.287825,0.325498,0.72983,0.589106,0.426227,0.209498


---------------------------------------------------------------------------

PerformanceMetric03 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.003534,0.004629,0.002409,0.003861,0.002969,0.003612
benchmark,0.002952,0.003834,0.002553,0.003561,0.002551,0.001604
is_greater_prob,0.494188,0.50716,0.489772,0.485683,0.479904,0.508117
is_pos_tse_ok,True,True,False,True,True,True
is_greater_prob_ok,False,True,False,False,False,True
is_performance_ok,False,True,False,False,False,True
Positive_tsa_std,0.044526,0.056435,0.038178,0.042248,0.03854,0.03976
benchmark_std,0.052104,0.075267,0.038572,0.043485,0.041545,0.043096
statistic,1.25916,0.628128,-0.188495,0.33516,0.450315,1.900984
pvalue,0.10399,0.264967,0.574754,0.368756,0.326248,0.028676


---------------------------------------------------------------------------


In [9]:
print("=" * 90)
pe_4form_year = PerformanceEvaluations4FormYear(processed_4form_df)

# PerformanceMetric01 evaluation:
metrics_df_01_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric01,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation (Year): ")
display(metrics_df_01_year)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_02b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation (Year): ")
display(metrics_df_02_year)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_03b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation (Year): ")
display(metrics_df_03_year)
print("-" * 75)

print("=" * 90)


PerformanceMetric01 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.003432,0.005436,0.000141,0.003044,0.008058,0.004183,0.001103,0.0058,0.003192,0.000229,0.005156
benchmark,0.001884,0.003115,-0.0031,0.002111,0.004568,0.002647,-0.001127,0.004183,0.002248,-0.001223,0.002947
is_greater_prob,0.630588,0.586294,0.598997,0.496222,0.55611,0.558603,0.601467,0.548544,0.543269,0.568345,0.620525
is_pos_tse_ok,True,True,True,True,True,True,True,True,True,True,True
is_greater_prob_ok,True,True,True,False,True,True,True,True,True,True,True
is_performance_ok,True,True,True,False,True,True,True,True,True,True,True
Positive_tsa_std,0.007423,0.026128,0.02947,0.019872,0.019575,0.018796,0.021101,0.022143,0.015606,0.021384,0.020383
benchmark_std,0.003258,0.012186,0.01395,0.009344,0.008095,0.008175,0.009256,0.012409,0.009155,0.011312,0.010195
statistic,3.936642,1.598512,1.985833,0.845936,3.299601,1.501143,1.957218,1.292948,1.064114,1.225532,1.983911
pvalue,0.000046,0.055248,0.023766,0.198974,0.000516,0.066948,0.025409,0.098246,0.14383,0.110416,0.023855


---------------------------------------------------------------------------

PerformanceMetric02 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.003432,0.005436,0.000141,0.003044,0.008058,0.004183,0.001103,0.0058,0.003192,0.000229,0.005156
benchmark,0.003088,0.005247,0.00298,0.004294,0.005791,0.004314,-0.000375,0.004322,0.004866,-0.000953,0.00558
is_greater_prob,0.552941,0.540609,0.481203,0.453401,0.513716,0.506234,0.515892,0.541262,0.485577,0.508393,0.49642
is_pos_tse_ok,True,True,False,False,True,False,True,True,False,True,False
is_greater_prob_ok,True,True,False,False,True,True,True,True,False,True,False
is_performance_ok,True,True,False,False,True,False,True,True,False,True,False
Positive_tsa_std,0.007423,0.026128,0.02947,0.019872,0.019575,0.018796,0.021101,0.022143,0.015606,0.021384,0.020383
benchmark_std,0.010283,0.025463,0.044979,0.020528,0.022105,0.018934,0.022214,0.020246,0.018436,0.02853,0.022282
statistic,0.559988,0.103271,-1.054587,-0.872235,1.537656,-0.097795,0.975368,0.999526,-1.413358,0.676868,-0.287409
pvalue,0.287825,0.458887,0.854007,0.808328,0.062267,0.53894,0.164834,0.158918,0.921032,0.249346,0.613065


---------------------------------------------------------------------------

PerformanceMetric03 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.003534,0.006827,0.000975,0.002021,0.006676,0.004597,0.001368,0.004372,0.002871,0.000967,0.004732
benchmark,0.002952,0.00434,0.001679,0.004085,0.005779,0.004489,-0.001485,0.00324,0.003517,-0.000755,0.004872
is_greater_prob,0.494188,0.500212,0.491768,0.463754,0.507744,0.487793,0.501174,0.512893,0.475615,0.505409,0.493929
is_pos_tse_ok,True,True,False,False,True,True,True,True,False,True,False
is_greater_prob_ok,False,True,False,False,True,False,True,True,False,True,False
is_performance_ok,False,True,False,False,True,False,True,True,False,True,False
Positive_tsa_std,0.044526,0.04503,0.082952,0.038176,0.031481,0.030643,0.037335,0.043337,0.028652,0.038674,0.03897
benchmark_std,0.052104,0.045322,0.110175,0.038752,0.03518,0.030774,0.045417,0.041308,0.032121,0.046725,0.039742
statistic,1.25916,1.891378,-0.245165,-1.731574,0.863694,0.1134,2.239162,0.873337,-0.695521,1.364898,-0.121209
pvalue,0.10399,0.029318,0.59683,0.958288,0.193903,0.454859,0.012599,0.191264,0.756617,0.086177,0.548234


---------------------------------------------------------------------------


## OutLayers Rem Evaluation test

In [10]:
post_processed_4form_df = processed_4form_df.copy()
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df.my_derivative_types.isin(['AB', 'B'])]
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df['transactionSharesAdjust'] > 0]

In [13]:
def _ecdf(sample):
    sample = np.atleast_1d(sample)
    quantiles, counts = np.unique(sample, return_counts=True)
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob


def _get_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[np.logical_and(cumprob > 0.5 * ci, cumprob < 1. - 0.5 * ci)]

def _get_left_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[cumprob < 1. - ci]

quantiles, cumprob = _ecdf(post_processed_4form_df[column_label_eval].values)
results_dict = dict()
results_dict["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()

quantiles, cumprob = _ecdf(post_processed_4form_df[shifted_columns_label_eval].values)
results_dict_bench = dict()
results_dict_bench["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict_bench["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict_bench["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict_bench["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()

In [14]:
print("\n MostProb Scenarios results: \n")
display(pd.Series(results_dict))
print("\n MostProb Scenarios results Bench: \n")
display(pd.Series(results_dict_bench))


 MostProb Scenarios results: 



E[R|0.05<P(R)<0.95]    0.003502
E[R|0.10<P(R)<0.90]    0.003572
E[R|0.15<P(R)<0.85]    0.003643
E[R|0.20<P(R)<0.80]    0.003705
dtype: float64


 MostProb Scenarios results Bench: 



E[R|0.05<P(R)<0.95]    0.003012
E[R|0.10<P(R)<0.90]    0.003192
E[R|0.15<P(R)<0.85]    0.003364
E[R|0.20<P(R)<0.80]    0.003464
dtype: float64